Introduction

This section examines whether changes in marriage prevalence (share of women 15–49 who are married or in a union) are associated with changes in fertility (total fertility rate) across countries and over time (1970–2024). We analyze both cross-country patterns and within-country dynamics, using:

Pooled correlations (Pearson & Spearman) to capture overall association,

Within-country (demeaned) correlations to remove time-invariant country factors,

Lag tests (1–3 years) to see whether marriage movements precede fertility movements,

Partial correlations controlling for log(GDP per capita) to test independence from income,

First differences (Δ) to assess year-to-year co-movement.

We report effect sizes (r/ρ), p-values, and sample sizes (n), and note where results remain robust after common sense checks (lags, within-country, partial).

H0: There is no statistical association between marriage prevalence (share of women 15–49 married/in union) and fertility (TFR) across countries and over time.

In [23]:
# Setup & Data Prep

import numpy as np
import pandas as pd
import os, math
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sqlalchemy import create_engine

# 1) Load the unified panel from your SQLite DB
engine = create_engine("sqlite:///analytics_panel.sqlite")
panel = pd.read_sql("SELECT * FROM v_panel_all;", con=engine)

# 2) Basic hygiene: build log GDP per capita once (for later partial tests)
panel["log_gdppc"] = np.log(panel["current_usd"]).replace([np.inf, -np.inf], np.nan)

# 3) Build the Marriage slice (years explicitly limited to 1970–2024, per your plan)
mar = (
    panel[["iso_code","year","fertility_rate","married_percentage","log_gdppc"]]
    .query("year >= 1970 and year <= 2024")
    .dropna(subset=["fertility_rate","married_percentage"])
    .copy()
)

# 4) Quick sanity checks you’ll cite in the write-up
summary = {
    "rows_in_panel": len(panel),
    "rows_in_marriage_slice": len(mar),
    "date_range_panel": (int(panel["year"].min()), int(panel["year"].max())),
    "date_range_marriage": (int(mar["year"].min()), int(mar["year"].max())),
    "non_null_fraction_fertility": round(panel["fertility_rate"].notna().mean()*100, 1),
    "non_null_fraction_married": round(panel["married_percentage"].notna().mean()*100, 1),
}

summary


{'rows_in_panel': 17927,
 'rows_in_marriage_slice': 11266,
 'date_range_panel': (1960, 2024),
 'date_range_marriage': (1970, 2023),
 'non_null_fraction_fertility': 94.4,
 'non_null_fraction_married': 68.1}

In [24]:
# Coverage counts for fertility & marriage 

#  ANY fertility & ANY marriage in the full panel (all years)
n_fert_any_panel = panel.loc[panel["fertility_rate"].notna(), "iso_code"].nunique(dropna=True)
n_mar_any_panel  = panel.loc[panel["married_percentage"].notna(), "iso_code"].nunique(dropna=True)

# ANY fertility & ANY marriage within 1970–2024 window (to match your analysis horizon)
mask_7024 = panel["year"].between(1970, 2024)
n_fert_7024 = panel.loc[mask_7024 & panel["fertility_rate"].notna(), "iso_code"].nunique(dropna=True)
n_mar_7024  = panel.loc[mask_7024 & panel["married_percentage"].notna(), "iso_code"].nunique(dropna=True)

# BOTH fertility & marriage non-null in the same rows (your `mar` slice already enforces this)
n_both_7024 = mar["iso_code"].nunique(dropna=True)

print("Unique countries with ANY fertility (all years):        ", n_fert_any_panel)
print("Unique countries with ANY marriage (all years):         ", n_mar_any_panel)
print("Unique countries with ANY fertility (1970–2024):        ", n_fert_7024)
print("Unique countries with ANY marriage (1970–2024):         ", n_mar_7024)
print("Unique countries with BOTH (1970–2024; your mar slice): ", n_both_7024)

# quick depth checks (years with data per country)
per_country_years = (
    panel.loc[mask_7024, ["iso_code","year","fertility_rate","married_percentage"]]
         .assign(has_fert=lambda d: d["fertility_rate"].notna().astype(int),
                 has_mar =lambda d: d["married_percentage"].notna().astype(int))
         .groupby("iso_code", dropna=True)
         .agg(years_fert=("has_fert", "sum"),
              years_mar =("has_mar", "sum"))
         .sort_values(["years_fert","years_mar"], ascending=[True, True])
)

# (uncomment to peek)
# per_country_years.head(10)


Unique countries with ANY fertility (all years):         265
Unique countries with ANY marriage (all years):          222
Unique countries with ANY fertility (1970–2024):         265
Unique countries with ANY marriage (1970–2024):          222
Unique countries with BOTH (1970–2024; your mar slice):  209


In [25]:
# Pooled correlations (1970–2024 window already applied in M1) 

def corr_tests(df, x="married_percentage", y="fertility_rate"):
    sub = df[[y, x]].dropna()
    n = len(sub)
    if n < 10:
        return pd.DataFrame([{"n": n, "pearson_r": np.nan, "pearson_p": np.nan,
                              "spearman_r": np.nan, "spearman_p": np.nan}])
    r_p, p_p = pearsonr(sub[y], sub[x])
    r_s, p_s = spearmanr(sub[y], sub[x])
    return pd.DataFrame([{"n": n, "pearson_r": float(r_p), "pearson_p": float(p_p),
                          "spearman_r": float(r_s), "spearman_p": float(p_s)}])

pooled_sig_marriage = corr_tests(mar)
pooled_sig_marriage


,n,pearson_r,pearson_p,spearman_r,spearman_p
0,11266,0.426064,0.0,0.384271,0.0


### Marriage ↔ Fertility — Pooled (1970–2024)

Interpretation: There is a moderate positive association between marriage prevalence and fertility in pooled cross-country/year data. Roughly 18% of fertility variation is associated with marriage prevalence in a linear sense (r² ≈ 0.426² ≈ **0.18**); the rank-based result (ρ² ≈ 0.15) confirms the pattern is monotonic.

In [26]:
# Within-country (demeaned) correlations 
def demean_by_country(df, cols=("fertility_rate","married_percentage")):
    out = df.copy()
    for c in cols:
        out[c] = out[c] - out.groupby("iso_code")[c].transform("mean")
    return out

mar_w = demean_by_country(mar)
within_sig_marriage = corr_tests(mar_w)
within_sig_marriage


,n,pearson_r,pearson_p,spearman_r,spearman_p
0,11266,0.454988,0.0,0.488411,0.0


After removing time-invariant country differences, fertility rises and falls with marriage prevalence. Effect size is large for macro panel work (r² ≈ 0.455² ≈ 0.21; ρ² ≈ 0.488² ≈ 0.24), indicating substantial within-country co-movement over time—not just cross-country level differences.

Why this matters: Unlike politics/safety (which mostly vanished within-country), marriage remains strongly associated with fertility inside countries, making it a leading candidate for your “strongest relationship” claim.

In [27]:
# Lag 

def corr_tests_cols(df, xcol, ycol="fertility_rate"):
    sub = df[[ycol, xcol]].dropna()
    n = len(sub)
    if n < 10:
        return {"n": n, "pearson_r": np.nan, "pearson_p": np.nan,
                "spearman_r": np.nan, "spearman_p": np.nan}
    r_p, p_p = pearsonr(sub[ycol].to_numpy(), sub[xcol].to_numpy())
    r_s, p_s = spearmanr(sub[ycol].to_numpy(), sub[xcol].to_numpy())
    return {"n": n, "pearson_r": float(r_p), "pearson_p": float(p_p),
            "spearman_r": float(r_s), "spearman_p": float(p_s)}

# Build lags without renaming anything
mar_l = mar.sort_values(["iso_code","year"]).copy()
for k in (1, 2, 3):
    mar_l[f"married_lag{k}"] = mar_l.groupby("iso_code")["married_percentage"].shift(k)

# Compute correlations using the explicit x column (no renaming)
lag_rows = []
for k in (1, 2, 3):
    x = f"married_lag{k}"
    stats = corr_tests_cols(mar_l, xcol=x, ycol="fertility_rate")
    stats["var"] = x
    lag_rows.append(stats)

lags_sig_marriage = pd.DataFrame(lag_rows)[["var","n","pearson_r","pearson_p","spearman_r","spearman_p"]]
lags_sig_marriage


,var,n,pearson_r,pearson_p,spearman_r,spearman_p
0,married_lag1,11057,0.429343,0.0,0.383613,0.0
1,married_lag2,10848,0.432409,0.0,0.382666,0.0
2,married_lag3,10639,0.435288,0.0,0.381610,0.0


Marriage prevalence leads fertility with a stable, moderate positive association across 1–3-year lags (r ≈ 0.43). Rank-based results (ρ ≈ 0.38) confirm the pattern is monotonic, not driven by linear outliers. Combined with the within-country result (r ≈ 0.46), this is strong evidence that changes in marriage rates are closely tied to changes in fertility over time.

In [28]:
# Partial correlation | log(GDPpc) 
def partial_corr_log_gdppc(slice_df, x="married_percentage", y="fertility_rate"):
    m = slice_df[["iso_code","year",y,x,"log_gdppc"]].dropna()
    n = len(m)
    if n < 10:
        return pd.DataFrame([{"n": n, "partial_r": np.nan, "partial_p": np.nan}])
    # residualize y ~ log_gdppc
    b1y, b0y = np.polyfit(m["log_gdppc"], m[y], 1); y_res = m[y] - (b1y*m["log_gdppc"] + b0y)
    # residualize x ~ log_gdppc
    b1x, b0x = np.polyfit(m["log_gdppc"], m[x], 1); x_res = m[x] - (b1x*m["log_gdppc"] + b0x)
    from scipy.stats import pearsonr
    r, p = pearsonr(y_res, x_res)
    return pd.DataFrame([{"n": n, "partial_r": float(r), "partial_p": float(p)}])

partial_sig_marriage = partial_corr_log_gdppc(mar)
partial_sig_marriage


,n,partial_r,partial_p
0,9941,0.187086,5.442435e-79


Even after holding income level (log GDP per capita) constant, marriage prevalence remains positively associated with fertility. The effect is small-to-moderate in magnitude (partial r ≈ 0.19), but with very strong statistical support given the large sample. This confirms that the marriage–fertility link is not just an artifact of income differences and helps explain why marriage showed the strongest within-country association in earlier tests.

In [29]:
r, n = 0.187086, 9941
z = np.arctanh(r)
se = 1/np.sqrt(n-3)
lo, hi = np.tanh(z + np.array([-1, 1]) * 1.96 * se)
(lo, hi)

(0.1680455322441913, 0.20598692567757368)

reject H0; marriage retains a positive, statistically robust association with fertility after controlling for income.

In [30]:
def corr_tests_cols(df, xcol, ycol):
    sub = df[[ycol, xcol]].dropna()
    n = len(sub)
    if n < 10:
        return pd.DataFrame([{"n": n, "pearson_r": np.nan, "pearson_p": np.nan,
                              "spearman_r": np.nan, "spearman_p": np.nan}])
    r_p, p_p = pearsonr(sub[ycol].to_numpy(), sub[xcol].to_numpy())
    r_s, p_s = spearmanr(sub[ycol].to_numpy(), sub[xcol].to_numpy())
    return pd.DataFrame([{"n": n, "pearson_r": float(r_p), "pearson_p": float(p_p),
                          "spearman_r": float(r_s), "spearman_p": float(p_s)}])

# Build first differences without renaming anything
mar_d = mar.sort_values(["iso_code","year"]).copy()
mar_d["d_fertility"] = mar_d.groupby("iso_code")["fertility_rate"].diff()
mar_d["d_married"]   = mar_d.groupby("iso_code")["married_percentage"].diff()

# Correlation on deltas
delta_sig_marriage = corr_tests_cols(mar_d, xcol="d_married", ycol="d_fertility")
delta_sig_marriage



,n,pearson_r,pearson_p,spearman_r,spearman_p
0,11057,0.098511,2.970106e-25,0.08583,1.554516e-19


In [31]:
os.makedirs("figs_marriage", exist_ok=True)

def _pearsonr(x, y):
    x = np.asarray(x, float); y = np.asarray(y, float)
    if x.size < 3 or y.size < 3: return float("nan")
    xm, ym = x - x.mean(), y - y.mean()
    den = np.sqrt((xm**2).sum()) * np.sqrt((ym**2).sum())
    if den == 0: return float("nan")
    return float((xm*ym).sum() / den)

def _ols(x, y):
    x = np.asarray(x, float); y = np.asarray(y, float)
    if x.size < 2: return float("nan"), float("nan")
    X = np.vstack([x, np.ones_like(x)]).T
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    return float(beta[0]), float(beta[1])

def _scatter_with_fit(x, y, title, xlabel, ylabel, savepath):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x, y, alpha=0.6)
    m, b = _ols(x, y)
    if not (np.isnan(m) or np.isnan(b)):
        xs = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        ax.plot(xs, m*xs + b, linewidth=2)
    ax.set_title(title)
    ax.set_xlabel(xlabel); ax.set_ylabel(ylabel)
    fig.tight_layout(); fig.savefig(savepath, dpi=160); plt.close(fig)
    return {"r": _pearsonr(x, y), "slope": m, "intercept": b}

# pooled
stats_pooled = _scatter_with_fit(
    x=mar["married_percentage"].to_numpy(),
    y=mar["fertility_rate"].to_numpy(),
    title="Fertility vs Marriage (pooled, 1970–2024)",
    xlabel="Married % (population)",
    ylabel="Fertility rate",
    savepath="figs_marriage/pooled.png"
)
print("Pooled:", stats_pooled)

# Within country (demeaned)
mar_w = mar.dropna(subset=["iso_code","married_percentage","fertility_rate"]).copy()
mar_w["_x"] = mar_w["married_percentage"] - mar_w.groupby("iso_code")["married_percentage"].transform("mean")
mar_w["_y"] = mar_w["fertility_rate"]      - mar_w.groupby("iso_code")["fertility_rate"].transform("mean")

stats_within = _scatter_with_fit(
    x=mar_w["_x"].to_numpy(),
    y=mar_w["_y"].to_numpy(),
    title="Fertility vs Marriage (within-country demeaned, 1970–2024)",
    xlabel="Married % (demeaned within country)",
    ylabel="Fertility rate (demeaned within country)",
    savepath="figs_marriage/within.png"
)
print("Within-country:", stats_within)

# Lag
mar_l = mar.sort_values(["iso_code","year"]).copy()
lag_results = []
for L in [1,2,3]:
    g = mar_l.groupby("iso_code", group_keys=False).apply(
        lambda d: d.assign(xlag=d["married_percentage"].shift(L))
    )
    g = g.dropna(subset=["xlag","fertility_rate"])
    stats_L = _scatter_with_fit(
        x=g["xlag"].to_numpy(),
        y=g["fertility_rate"].to_numpy(),
        title=f"Fertility_t vs Marriage_(t-{L}) (pooled, 1970–2024)",
        xlabel=f"Married % (t-{L})",
        ylabel="Fertility rate (t)",
        savepath=f"figs_marriage/lag{L}.png"
    )
    stats_L["lag"] = L; stats_L["n"] = int(g.shape[0])
    lag_results.append(stats_L)

pd.DataFrame(lag_results)[["lag","n","r","slope","intercept"]]


Pooled: {'r': 0.42606392780956603, 'slope': 0.08726212654621349, 'intercept': -1.6530767725754028}
Within-country: {'r': 0.45498830413048325, 'slope': 0.11372688070129403, 'intercept': -1.3911597290491914e-16}


,lag,n,r,slope,intercept
0,1,11057,0.429343,0.087544,-1.705906
1,2,10848,0.432409,0.087751,-1.754225
2,3,10639,0.435288,0.087886,-1.797890


### Marriage and Fertility — First Differences, Cell M6

**Decision:** All p < 0.05: reject H0.

**Interpretation:** Year-to-year increases in marriage prevalence are associated with increases in fertility. The effect size is small (r ≈ 0.10), but statistically precise given large n. Together with the within-country (demeaned) result (r ≈ 0.455) and lagged results (r ≈ 0.43 for k=1–3), this provides a consistent picture: marriage prevalence co-moves with fertility both in levels (within countries) and in short-run changes.

---

## Summary 

* **Pooled** r ≈ 0.43, ρ ≈ 0.38 **reject H0**
* **Within-country (demeaned):** r ≈ 0.455, ρ ≈ 0.488 **reject H0**
* **Lags (k=1–3):** r ≈ 0.43 each **reject H0**
* **Partial | log(GDPpc):** partial r ≈ 0.187 p ≈ 0.000 **reject H0**
* **First differences (delta):** r ≈ 0.10, ρ ≈ 0.086  **reject H0**

**Conclusion:** Marriage prevalence shows the strongest and most robust association with fertility across all checks (pooled, within-country, lagged, partial, and delta). While short-run effects are modest, the within-country and partial results demonstrate a substantive, positive link that persists beyond income differences.
